

- final file has some duplicate values, fix them (fixed in analysis)
(154	behaviour	12789	4.59	5	1175	0.12 and
155	behaviour	12789	4.59	5	1175	0.12)


C5-NC5 indicates that the MI-score with the cut-off point three was used as the association measure (IN SCRIPT); the collocates were identified in the span of three words to the left (L3) and three words to the right (R3) of the node and the frequency threshold was five for both the collocate (C5) and the collocation (NC5). (frequency threshold exists)

Generally, the smaller the span, the greater the focus of the analysis on the most immediate lexico-grammatical patterns; a larger span captures looser associations.

TO DO
- make the freq threshold vary based on the amount of data for each year gap!!! (right now its 5 always) (make it scale up) (differently per gap, based on how many characters per year) (kinda done, under testing)
- rerun with 3l3r (done)
- avoid lemmatization but tokenize (list of fact words) (fact/facts) (done)
- include as a side if its from rstl rstb or rsta (not done)
- split into two periods to avoid the gap (done)

4.5m words or less = 5 freq treshold for 1665 to 1958
for every 900k words it adds 1

5m words or less = 5 freq treshold for 2000 to 2024
for every 1m add 1


still need to experiment with this!

TESTING CONTEXTS

In [20]:
import os
import pandas as pd

folder_path = "collocate_results_combined_fact_10_window/csv_ver"

csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
combined_data = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

unique_words = combined_data['Word'].unique()
print(unique_words)

keyword = "lsw"
keyword_docs = []

for f in csv_files:
    df = pd.read_csv(f)
    if keyword in df['Word'].values:
        keyword_docs.append(f)

if keyword_docs:
    print(f"The keyword '{keyword}' is found in the following document(s):")
    for doc in keyword_docs:
        print(doc)
else:
    print(f"The keyword '{keyword}' is not found in any document.")

['matter' 'truth' 'certainty' ... 'fitness' 'affect' 'society']
The keyword 'lsw' is found in the following document(s):
collocate_results_combined_fact_10_window/csv_ver\collocates_2013-2022.csv
collocate_results_combined_fact_10_window/csv_ver\collocates_2014-2023.csv
collocate_results_combined_fact_10_window/csv_ver\collocates_2015-2024.csv


In [21]:
import pandas as pd
import nltk
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import word_tokenize
import string
import csv
import os
import re
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import traceback

nltk.download('punkt_tab')

def extract_year_from_filename(filename):
    match = re.search(r'rst[bla]?_?(\d{4})', filename)
    return int(match.group(1)) if match else None

def process_file(file_path):
    try:
        year = extract_year_from_filename(os.path.basename(file_path))
        if not year:
            return None
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        text = text.replace('ſ', 's').replace('Å¿', 's').replace('obseryed', 'observed').replace('thab', 'that')
        return year, text
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None

def process_tokens(text):
    tokens = word_tokenize(text.lower())
    return [token for token in tokens 
            if token.isalpha() and len(token) > 1 and token not in string.punctuation]

def calculate_scaled_freq_filter(total_words, time_period):
    """Calculate frequency filter threshold based on word count and time period"""
    if time_period == "historical":  # 1665-1958
        base_threshold = 5
        if total_words <= 4500000:
            return base_threshold
        return base_threshold + ((total_words - 4500000) // 900000)
    else:  # 2000-2024
        base_threshold = 5
        if total_words <= 5000000:
            return base_threshold
        return base_threshold + ((total_words - 5000000) // 1000000)

def process_window(window_data, word_of_interest, time_period, collocate_window):
    try:
        window_start, window_end, window_texts = window_data
        all_tokens = [token for text in window_texts for token in process_tokens(text)]
        
        total_tokens = len(all_tokens)
        total_texts = len(window_texts)
        
        # Calculate scaled frequency filter based on total words
        freq_filter = calculate_scaled_freq_filter(total_tokens, time_period)
        print(f"Window {window_start}-{window_end}: Using frequency filter of {freq_filter} for {total_tokens} total words")
        
        finder = BigramCollocationFinder.from_words(all_tokens)
        finder.apply_freq_filter(freq_filter)
        
        collocations = finder.score_ngrams(BigramAssocMeasures().pmi)
        
        collocate_stats = []
        collocate_contexts = defaultdict(list)

        word_of_interest_lower = word_of_interest.lower()
        word_of_interest_freq = sum(finder.word_fd[word] for word in finder.word_fd if word.lower() == word_of_interest_lower)

        for bigram, pmi in collocations:
            if word_of_interest_lower in (word.lower() for word in bigram):
                other_word = bigram[0] if bigram[1].lower() == word_of_interest_lower else bigram[1]
                observed_freq = finder.ngram_fd[bigram]
                word_freq = finder.word_fd[other_word]
                expected_freq = (word_of_interest_freq * word_freq) / total_tokens
                num_texts = sum(1 for text in window_texts if other_word.lower() in text.lower().split())
                
                if num_texts > 1:
                    collocate_stats.append({
                        'word': other_word,
                        'total_corpus': word_freq,
                        'expected_freq': expected_freq,
                        'observed_freq': observed_freq,
                        'num_texts': num_texts,
                        'pmi': pmi
                    })
                    
                    for text in window_texts:
                        words = text.split()
                        for i, word in enumerate(words):
                            if word.lower() == word_of_interest_lower and other_word.lower() in [w.lower() for w in words[max(0, i-collocate_window):i+collocate_window+1]]:
                                context = ' '.join(words[max(0, i-collocate_window):i+collocate_window+1])
                                collocate_contexts[other_word].append((f"Years {window_start}-{window_end}: {context}", other_word))
        
        collocate_stats.sort(key=lambda x: x['pmi'], reverse=True)
        top_collocates = collocate_stats
        
        filtered_contexts = [context for collocate in top_collocates for context in collocate_contexts.get(collocate['word'], [])]
        
        return f"{window_start}-{window_end}", {
            'collocates': top_collocates,
            'total_tokens': total_tokens,
            'total_texts': total_texts,
            'contexts': filtered_contexts
        }
    except Exception as e:
        print(f"Error processing window {window_start}-{window_end}: {str(e)}")
        traceback.print_exc()
        return None

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Igiba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import pandas as pd
import nltk
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import csv
import os
import re
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import traceback

def analyze_collocates(text_dirs, output_dir, word_of_interest, start_year, end_year, time_period, window_size=10, collocate_window=3):
    def read_texts_by_year(directories):
        texts_by_year = defaultdict(list)
        file_paths = [os.path.join(dir, f) for dir in directories for f in os.listdir(dir) if f.endswith('.txt')]
        
        print("Reading files...")
        for file_path in tqdm(file_paths):
            result = process_file(file_path)
            if result:
                year, text = result
                if start_year <= year <= end_year:  # Only include texts within specified year range
                    texts_by_year[year].append(text)
        
        return texts_by_year

    print(f"Starting text processing for years {start_year}-{end_year}...")
    texts_by_year = read_texts_by_year(text_dirs)
    
    all_years = sorted(texts_by_year.keys())
    if not all_years:
        print(f"No texts found in the specified year range {start_year}-{end_year}")
        return
        
    window_data = [
        (window_start, window_start + window_size - 1, 
         [text for year in range(window_start, window_start + window_size) for text in texts_by_year.get(year, [])])
        for window_start in range(start_year, end_year - window_size + 2)
    ]

    collocates_by_window = {}
    print("Processing windows...")
    for data in tqdm(window_data):
        result = process_window(data, word_of_interest, time_period, collocate_window)
        if result:
            window, window_result = result
            collocates_by_window[window] = window_result

    output_dir = f"{output_dir}_{start_year}_{end_year}"
    os.makedirs(output_dir, exist_ok=True)
    print("Saving results to CSV and TXT files...")

    for window, data in collocates_by_window.items():
        # Save CSV file
        csv_filename = os.path.join(output_dir, f'collocates_{window}.csv')
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['No.', 'Word', 'Total no. in window corpus', 'Expected collocate frequency',
                             'Observed collocate frequency', 'In no. of texts', 'Mutual Information value'])
            
            for i, collocate in enumerate(data['collocates'], 1):
                writer.writerow([
                    i,
                    collocate['word'],
                    collocate['total_corpus'],
                    f"{collocate['expected_freq']:.2f}",
                    collocate['observed_freq'],
                    collocate['num_texts'],
                    f"{collocate['pmi']:.2f}"
                ])

        txt_filename = os.path.join(output_dir, f'collocates_txt_{window}.txt')
        with open(txt_filename, 'w', encoding='utf-8') as txtfile:
            txtfile.write("https://example.com/collocation-analysis")
            txtfile.write(f"There are {data['total_tokens']} different words in your collocation database for \"{word_of_interest}\". ")
            txtfile.write(f"(Your query returned {sum(c['observed_freq'] for c in data['collocates'])} matches in {data['total_texts']} different texts)\n")
            txtfile.write("__________________\n\n")
            
            txtfile.write("No.\tWord\tTotal no. in whole corpus\tExpected collocate frequency\t")
            txtfile.write("Observed collocate frequency\tIn no. of texts\tMutual information value\n\n")
            
            for i, collocate in enumerate(data['collocates'], 1):
                txtfile.write(f"{i}\t{collocate['word']}\t{collocate['total_corpus']}\t")
                txtfile.write(f"{collocate['expected_freq']:.2f}\t{collocate['observed_freq']}\t")
                txtfile.write(f"{collocate['num_texts']}\t{collocate['pmi']:.2f}\n")

        context_file = os.path.join(output_dir, f'collocate_contexts_{window}.csv')
        with open(context_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Context', 'Collocate'])
            writer.writerows(data['contexts'])

    print(f"Collocate analysis completed. Results saved in the '{output_dir}' directory.")

#historical period (1665-1958)
analyze_collocates([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
                   "collocate_results_1665-1958_fact", "fact", 1665, 1958, "historical")

analyze_collocates([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
                   "collocate_results_1665-1958_facts", "facts", 1665, 1958, "historical")

#modern period (2000-2024)
analyze_collocates([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
                   "collocate_results_2000-2024_fact", "fact", 2000, 2024, "modern")

analyze_collocates([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
                   r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
                   "collocate_results_2000-2024_facts", "facts", 2000, 2024, "modern")

Starting text processing for years 1665-1958...
Reading files...


  0%|          | 0/18194 [00:00<?, ?it/s]

Processing windows...


  0%|          | 0/285 [00:00<?, ?it/s]

Window 1665-1674: Using frequency filter of 5 for 959179 total words
Window 1666-1675: Using frequency filter of 5 for 907674 total words
Window 1667-1676: Using frequency filter of 5 for 918367 total words
Window 1668-1677: Using frequency filter of 5 for 1016400 total words
Window 1669-1678: Using frequency filter of 5 for 919485 total words
Window 1670-1679: Using frequency filter of 5 for 832797 total words
Window 1671-1680: Using frequency filter of 5 for 707737 total words
Window 1672-1681: Using frequency filter of 5 for 598074 total words
Window 1673-1682: Using frequency filter of 5 for 497274 total words
Window 1674-1683: Using frequency filter of 5 for 526114 total words
Window 1675-1684: Using frequency filter of 5 for 538085 total words
Window 1676-1685: Using frequency filter of 5 for 552172 total words
Window 1677-1686: Using frequency filter of 5 for 601140 total words
Window 1678-1687: Using frequency filter of 5 for 503107 total words
Window 1679-1688: Using frequency

---
extra stuff
---

In [42]:
#so my pc turns off eventually
import os
import time

time.sleep(100)
os.system("shutdown /s /t 1")

0

In [1]:
#checking delimiter type
import re
from collections import defaultdict

def get_file_delimiter(file, check_lines=2):
    delimiters = [',', '\t', ';', '|', ':']
    results = defaultdict(int)

    with open(file, 'r') as f:
        for i, line in enumerate(f):
            if i > check_lines:
                break
            for delimiter in delimiters:
                # Use re.split to split based on the delimiter
                fields = re.split(re.escape(delimiter), line)
                if len(fields) > 1:
                    results[delimiter] += 1

    # Get the delimiter with the maximum count
    best_delimiter = max(results, key=results.get)
    return best_delimiter

# Example usage:
file_path = 'collocate_results_combined_fact2/collocates_2015-2019.csv'
delimiter = get_file_delimiter(file_path)
print(f"Detected delimiter: {delimiter}")


Detected delimiter: ,


In [3]:
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict
import os
import re
from tqdm import tqdm

def count_files_in_year_windows(text_dirs, window_size=5):
    def extract_year_from_filename(filename):
        match = re.search(r'rst[bla]?_?(\d{4})', filename)
        return int(match.group(1)) if match else None

    year_counts = defaultdict(int)

    print("Counting files in year windows...")
    file_paths = [os.path.join(dir, f) for dir in text_dirs for f in os.listdir(dir) if f.endswith('.txt')]

    for file_path in tqdm(file_paths):
        year = extract_year_from_filename(os.path.basename(file_path))
        if year:
            year_counts[year] += 1

    all_years = sorted(year_counts.keys())
    year_windows = defaultdict(int)

    for year in all_years:
        if year in year_counts:
            for offset in range(window_size):
                window_year = year + offset
                year_windows[window_year] += year_counts[year] if window_year in year_counts else 0

    return dict(year_windows)

# Example usage
year_counts = count_files_in_year_windows(
    [r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
     r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
     r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"],
    window_size=5
)

# Prepare the data for plotting
years = list(range(1665, 2025))
counts = [year_counts.get(year, 0) for year in years]
df = pd.DataFrame({'Year': years, 'Article Count': counts})

# Create the interactive plot
fig = go.Figure()

# Add scatter points for each year
fig.add_trace(go.Scatter(
    x=df['Year'],
    y=df['Article Count'],
    mode='markers+lines',
    marker=dict(
        size=10,
        color=['red' if count == 0 else 'orange' if count < 100 else 'blue' for count in df['Article Count']],
        line=dict(width=1, color='black')
    ),
    text=[f'Year: {year}<br>Count: {count}' for year, count in zip(df['Year'], df['Article Count'])],  # Tooltip text
    hoverinfo='text'  # Show the text on hover
))

# Update layout
fig.update_layout(
    title='Number of Articles per Year (1665-2024) (without restricteds)',
    xaxis_title='Year',
    yaxis_title='Number of Articles',
    xaxis=dict(
        tickmode='linear',
        tick0=1665,
        dtick=5,
        range=[1665, 2024]  # Explicitly set the range of the x-axis
    ),
    yaxis=dict(range=[0, df['Article Count'].max() + 500]),  # Adjust the limit as necessary
    template='plotly_white'
)

# Show the plot
fig.show()


Counting files in year windows...


100%|██████████| 18194/18194 [00:00<00:00, 184788.91it/s]


In [19]:
def calculate_words_per_10_year_window(year_word_counts, start_year, end_year):
    words_per_window = {}
    for year in range(start_year, end_year - 9):
        window_start = year
        window_end = year + 9
        total_words = sum(year_word_counts.get(y, 0) for y in range(window_start, window_end + 1))
        words_per_window[f"{window_start}-{window_end}"] = total_words
    return words_per_window

# Calculate words for the historical period (1665-1958)
historical_words_per_window = calculate_words_per_10_year_window(year_word_counts, 1665, 1958)

# Calculate words for the modern period (2000-2024)
modern_words_per_window = calculate_words_per_10_year_window(year_word_counts, 2000, 2024)

print("Words per 10-year window (1665-1958):")
for window, total_words in historical_words_per_window.items():
    print(f"{window}: {total_words}")

print("\nWords per 10-year window (2000-2024):")
for window, total_words in modern_words_per_window.items():
    print(f"{window}: {total_words}")

# Prepare data for plotting
historical_windows = list(historical_words_per_window.keys())
historical_counts = list(historical_words_per_window.values())
modern_windows = list(modern_words_per_window.keys())
modern_counts = list(modern_words_per_window.values())

# Create the plot for the historical period
fig_historical = go.Figure()

fig_historical.add_trace(go.Bar(
    x=historical_windows,
    y=historical_counts,
    name='Historical Period (1665-1958)',
    marker=dict(color='blue'),
    text=historical_counts,
    textposition='auto',
    hovertemplate='Window: %{x}<br>Words: %{y}<extra></extra>'
))

fig_historical.update_layout(
    title='Total Words per 10-Year Window (1665-1958)',
    xaxis_title='10-Year Window',
    yaxis_title='Total Words',
    template='plotly_white'
)

fig_historical.show()

# Create the plot for the modern period
fig_modern = go.Figure()

fig_modern.add_trace(go.Bar(
    x=modern_windows,
    y=modern_counts,
    name='Modern Period (2000-2024)',
    marker=dict(color='orange'),
    text=modern_counts,
    textposition='auto',
    hovertemplate='Window: %{x}<br>Words: %{y}<extra></extra>'
))

fig_modern.update_layout(
    title='Total Words per 10-Year Window (2000-2024)',
    xaxis_title='10-Year Window',
    yaxis_title='Total Words',
    template='plotly_white'
)

fig_modern.show()

Words per 10-year window (1665-1958):
1665-1674: 4483034
1666-1675: 4879216
1667-1676: 5157852
1668-1677: 5431479
1669-1678: 5488557
1670-1679: 5326316
1671-1680: 5083428
1672-1681: 4701344
1673-1682: 4088247
1674-1683: 3631576
1675-1684: 3301432
1676-1685: 3139347
1677-1686: 3171548
1678-1687: 3208758
1679-1688: 3205762
1680-1689: 3181589
1681-1690: 3134502
1682-1691: 3021500
1683-1692: 3021500
1684-1693: 3030973
1685-1694: 2993241
1686-1695: 3058683
1687-1696: 2939697
1688-1697: 2820711
1689-1698: 2843882
1690-1699: 3064049
1691-1700: 3394498
1692-1701: 3909375
1693-1702: 4590675
1694-1703: 4970337
1695-1704: 5336541
1696-1705: 5241924
1697-1706: 5245593
1698-1707: 5082839
1699-1708: 5065230
1700-1709: 4761497
1701-1710: 4676151
1702-1711: 4492519
1703-1712: 4142464
1704-1713: 3884136
1705-1714: 3723251
1706-1715: 3591344
1707-1716: 3361151
1708-1717: 3458944
1709-1718: 3323106
1710-1719: 3239616
1711-1720: 3048013
1712-1721: 2954696
1713-1722: 2826372
1714-1723: 2757951
1715-1724: 2

In [4]:
import pandas as pd
import plotly.graph_objects as go
import os

def find_highest_numbers_in_directory(directory_path):
    highest_numbers = {}

    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    highest_number = 0  # Initialize highest number for the current file
                    for line in f.readlines()[1:]:  # Skip the header line
                        parts = line.split('\t')
                        if len(parts) > 0:  # Check if the line is properly formatted
                            try:
                                number = int(parts[0])  # Get the number from the first column
                                highest_number = max(highest_number, number)  # Update highest number
                            except ValueError:
                                pass  # Ignore non-integer values
                    
                    # Store the highest number for the current file
                    highest_numbers[filename] = highest_number

            except PermissionError:
                pass  # Ignore permission errors
            except FileNotFoundError:
                pass  # Ignore file not found errors
            except Exception:
                pass  # Ignore all other exceptions

    return highest_numbers

# Example usage
collocate_directory_path = r"collocate_results_combined_fact_10_window"
highest_numbers = find_highest_numbers_in_directory(collocate_directory_path)

# Prepare the data for plotting
df = pd.DataFrame(list(highest_numbers.items()), columns=['File', 'Highest Number'])

# Modify the 'File' column to extract just the year range
df['Year Range'] = df['File'].str.extract(r'(\d{4}-\d{4})')[0]  # Extract year range using regex
df['Year Range'] = df['Year Range'].fillna(df['File'])  # In case of no match, keep the original filename

# Create the interactive plot
fig = go.Figure()

# Add bars for each file's highest number
fig.add_trace(go.Bar(
    x=df['Year Range'],  # Use the modified Year Range for x-axis
    y=df['Highest Number'],
    marker=dict(color='blue'),
    text=df['Highest Number'],  # Show counts on bars
    textposition='auto',  # Position text on the bars
    hovertemplate='Year Range: %{x}<br>%{y}<extra></extra>'  # Customize hover text
))

# Update layout
fig.update_layout(
    title='Amount of Collocates per Year Window',
    xaxis_title='Year Range',
    yaxis_title='Highest Number',
    xaxis_tickangle=-45,  # Rotate x-axis labels for better visibility
    template='plotly_white'
)

# Show the plot
fig.show()


In [6]:
#small mistake, fixing all header to line 1
import os

folder_path = 'collocate_results_combined_fact_10_window/'

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r') as file:
            lines = file.readlines()

        if len(lines) > 1:
            combined_first_two = lines[0].strip() + " " + lines[1].strip() + '\n'
            lines = [combined_first_two] + lines[2:]

        with open(file_path, 'w') as file:
            file.writelines(lines)
